In [1]:
import numpy as np
import json
import random
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from typing import Tuple, List, Any
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

# Load Data
Using the subset data. 1000 user for now

In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
%cd ./Neural-CF

/content/drive/MyDrive/Neural-CF


In [4]:
def load_data(
    data_folder: str,
    filenames=("yelp_academic_dataset_user.json", "yelp_academic_dataset_business.json", "yelp_academic_dataset_review.json")
    ) -> Tuple[List[Any], ...]:
    return tuple(map(
                  lambda f: list(map(json.loads, open(os.path.join(data_folder, f), "r", encoding="utf-8").readlines())),
                  filenames
                ))

(subset_user_data, subset_business_data, subset_review_data) = load_data("./Yelp-Dataset/subset",
                                                                        ("subset_user.json",
                                                                          "subset_business.json",
                                                                          "subset_review.json"))

In [5]:
print(subset_user_data[0].keys())
print(subset_business_data[0].keys())
print(subset_review_data[0].keys())

dict_keys(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny', 'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos'])
dict_keys(['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours'])
dict_keys(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date'])


In [6]:
# Display City names in the subset data
city_list = []
for i, business in enumerate(subset_business_data):
  if business['city'] not in city_list:
    city_list.append(business['city'])
print(city_list)

['Philadelphia', 'Tampa', 'St Louis', 'St. Louis', 'New Orleans', 'Tucson', 'Saint Louis', 'Indianapolis', 'Ardmore', 'Collingswood', 'Dunedin', 'Tarpon Springs', 'Saint Petersburg', 'Cherry Hill', 'Nashville', 'Reno', 'St. Petersburg', 'Clayton', 'Brandon', 'East Norriton', 'Pottstown', 'Tierra Verde', 'Lutz', 'Brentwood', 'Oldsmar', 'Kennett Square', 'Media', 'King Of Prussia', 'Florissant', 'King of Prussia', 'Carmel', 'ST LOUIS', 'Conshohocken', 'Clearwater Beach', 'Wynnewood', 'Sparks', 'Cinnaminson', 'Bensalem', 'North Redington Bch', 'Clearwater', 'Alton', 'Camden', 'Plymouth Meeting', 'Westmont', 'Creve Coeur', 'Horsham', 'Palm Harbor', 'Harahan', 'Manayunk', 'Maplewood', 'Kenner', 'South Tampa', 'St Petersburg', 'Maple Shade', 'Webster Groves', 'Upper Darby', 'Fishers', 'Collegeville', 'Haddon Township', 'Downingtown', 'Chesterfield', 'Wayne', 'Plant City', 'Richmond Heights']


# Data Preprocess Test (For Debugging)
For debugging only, no need to run

Ref: https://github.com/zhrlove/NCF/tree/master \
Ref2: https://github.com/hexiangnan/sigir16-eals

NCF: https://github.com/hexiangnan/neural_collaborative_filtering \
NCF (torch): https://github.com/yihong-chen/neural-collaborative-filtering/tree/master

Training Input: `userID::itemID::rating::timestamp (if have)`

In [ ]:
for i, business in enumerate(subset_business_data):
  if business['business_id'] == 'gGyqnAlpFrka_qzpO7j4lQ':
    print(business['name'])
for i, user in enumerate(subset_user_data):
  if user['user_id'] == 'GcdYgbaF75vj7RO6EZhPOQ':
    print(user['name'])

Citizens Bank Park
Kathleen


In [ ]:
# Reindex
user_item_interactions = subset_review_data
df = pd.DataFrame(user_item_interactions)
print("Number of Unique Users:", df['user_id'].nunique())
print("Number of Unique Businesses:", df['business_id'].nunique())

df = df.groupby(['user_id', 'business_id']).agg({'stars': 'mean'}).reset_index()
print(df.head(10))

user_id = df[['user_id']].drop_duplicates().reindex()
user_id['userId'] = np.arange(len(user_id))
ml1m_rating = pd.merge(df, user_id, on=['user_id'], how='left')

item_id = df[['business_id']].drop_duplicates()
item_id['itemId'] = np.arange(len(item_id))
yelp_rating = pd.merge(ml1m_rating, item_id, on=['business_id'], how='left')
yelp_rating = yelp_rating[['userId', 'itemId', 'stars']]
print(yelp_rating.head(10))
print('Range of userId is [{}, {}]'.format(yelp_rating.userId.min(), yelp_rating.userId.max()))
print('Range of itemId is [{}, {}]'.format(yelp_rating.itemId.min(), yelp_rating.itemId.max()))



Number of Unique Users: 961
Number of Unique Businesses: 1000
                  user_id             business_id  stars
0  -3s52C4zL_DHRK0ULG6qtg  -kqjc8DxxRac4cz2qTKCLw    4.0
1  -3s52C4zL_DHRK0ULG6qtg  0QYWhij_YZ7Lyk9F6213Sg    5.0
2  -3s52C4zL_DHRK0ULG6qtg  1YflE3DkiCZGgLnf3paLnA    5.0
3  -3s52C4zL_DHRK0ULG6qtg  2BMk_drsikKWslJCXmQtjQ    2.0
4  -3s52C4zL_DHRK0ULG6qtg  2IahpaBR4U2Kdy9HF28EQA    2.5
5  -3s52C4zL_DHRK0ULG6qtg  33JlrWf0kmHd2VzW58Wp0g    3.0
6  -3s52C4zL_DHRK0ULG6qtg  6t0sNev22mcbvOB4gYVVOw    3.0
7  -3s52C4zL_DHRK0ULG6qtg  89SD5fNDDnJj-ITB40hLsQ    1.0
8  -3s52C4zL_DHRK0ULG6qtg  8O35ji_yOMVJmZ6bl96yhQ    3.0
9  -3s52C4zL_DHRK0ULG6qtg  8QZJvkx29OQNZgrM53aVbw    4.0
   userId  itemId  stars
0       0       0    4.0
1       0       1    5.0
2       0       2    5.0
3       0       3    2.0
4       0       4    2.5
5       0       5    3.0
6       0       6    3.0
7       0       7    1.0
8       0       8    3.0
9       0       9    4.0
Range of userId is [0, 960]
Range of

# Training Setup

We can train NeuralMF without training the GMF and MLP. But, the author suggest that training GMF and MLP first can lead to better performance for large scale data.

Edit the config parameter in `train.py` to adjust any hyperparameters

In [7]:
%cd ./Torch-NCF

/content/drive/MyDrive/Neural-CF/Torch-NCF


In [8]:
# Make directory to save the models
import os
if not os.path.exists('checkpoints'):
    os.makedirs('checkpoints')

# For running in Google Colab
!pip install tensorboardX==1.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.3/216.3 kB 5.3 MB/s eta 0:00:00


# Training GMF
Before Training, adjust the config for num_users and num_items correctly

In [ ]:
!python train.py --data_dir '/content/drive/MyDrive/Neural-CF/Yelp-Dataset/subset/subset_review.json' --model 'gmf'

# Training MLP
Set the pretrained MF path in the mlp_config in `train.py` if you set pretrain = true.


In [ ]:
!python train.py --data_dir '/content/drive/MyDrive/Neural-CF/Yelp-Dataset/subset/subset_review.json' --model 'mlp'

# Train NeuralMF
Edit the pretrain setting of neumf_config in `train.py` to determine whether you want to use the pretrained MLP and GMF to train the NeuralMF or not.

For small predictive factors, running NeuMF without pre-training can achieve better performance than GMF and MLP. For large predictive factors, pre-training NeuMF can yield better performance.

In [ ]:
!python train.py --data_dir '/content/drive/MyDrive/Neural-CF/Yelp-Dataset/subset/subset_review.json' --model 'neumf'

Loading Data....
/content/drive/MyDrive/Neural-CF/Torch-NCF/data.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['rating'][ratings['rating'] > 0] = 1.0
     userId  ...                                   negative_samples
0         0  ...  [922, 452, 834, 969, 488, 99, 323, 581, 555, 4...
1         1  ...  [840, 950, 559, 214, 992, 824, 623, 566, 607, ...
2         2  ...  [46, 871, 622, 684, 966, 993, 582, 651, 731, 1...
3         3  ...  [617, 750, 12, 511, 802, 80, 386, 799, 46, 600...
4         4  ...  [14, 784, 822, 2, 892, 741, 591, 676, 118, 245...
..      ...  ...                                                ...
956     956  ...  [999, 789, 465, 11, 443, 631, 249, 753, 152, 5...
957     957  ...  [262, 181, 633, 427, 504, 329, 284, 694, 877, ...
958     958  ...  [22, 499, 26, 303,

# Evaluated with Hit Ratio (HR) and Normalized Discounted Cumulative Gain (NDCG)

In [7]:
!lsof -i :6006

COMMAND    PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
tensorboa 4311 root   14u  IPv4 117764      0t0  TCP localhost:6006 (LISTEN)


In [8]:
!kill -9 4311

In [ ]:
# Load Tensorboard
%reload_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/Neural-CF/Torch-NCF/runs/pretrain_neumf_factor8neg4' --port 6006